In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = "1,2"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"
from trl import SFTTrainer, setup_chat_format
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq

from transformers import AutoTokenizer, AutoModelForCausalLM
from lib import Datasets
from datasets import load_dataset
import copy
from transformers import Trainer, TrainingArguments
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence, List, Literal

import torch
import transformers
from transformers import Trainer
from datasets import load_dataset

2025-02-15 20:53:17.226519: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-15 20:53:17.241964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739641997.258832  208926 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739641997.263961  208926 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-15 20:53:17.282510: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# name = "meta-llama/Llama-3.2-1B"
name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch.float16, device_map='balanced')
model = AutoModelForCausalLM.from_pretrained(name, device_map='cuda')

# model = get_peft_model(model, peft_config)

print(model)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [3]:
dataset = load_dataset("meta-math/MetaMathQA", split='train[:10]')
from transformers import Trainer, TrainingArguments, EvalPrediction
from sklearn.metrics import accuracy_score
import numpy as np
from evaluate import load

accuracy_metric = load('accuracy')


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(predictions.shape, labels.shape)
    sm = dict()
    sm['accuracy'] = 0
    k = labels.shape[0]
    
    for i in range(len(predictions)):

        mask = labels[i] != -100
        valid_labels = labels[i][mask]
        valid_predictions = predictions[i][mask]
    
        valid_predictions = np.roll(valid_predictions, 1)
    

        # Вычисление accuracy
        accuracy = accuracy_metric.compute(predictions=valid_predictions, references=valid_labels)
        print(accuracy)
        sm['accuracy'] += accuracy['accuracy'] / k
        
    return sm


In [4]:
PROMPT = (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response: Let's think step by step."
    )

In [5]:
# def format_and_tokenize(examples):
#     texts = [f"Question: {q}\nAnswer: {a}" for q, a in zip(examples['query'], examples['response'])]
#     t =  tokenizer(
#         texts,
#         truncation=True,
#         max_length=768,
#         padding="max_length",
#         return_tensors="pt"
#     )
#     t['attention_mask'][:50] = 0
#     return t

def format_and_tokenize(examples):
    texts = [PROMPT.format(instruction=q) for q, a in zip(examples['query'], examples['response'])]
    tks = tokenizer(
        texts,
        truncation=True,
        max_length=2048,
        padding="max_length",
        return_tensors="pt"
    )
    
    print(tks)
    
    lens = [len(t) for t in tks['input_ids']]
    texts = [PROMPT.format(instruction=q) + a for q, a in zip(examples['query'], examples['response'])]
    t =  tokenizer(
        texts,
        truncation=True,
        max_length=768,
        padding="max_length",
        return_tensors="pt"
    )
    for i in range(len(lens)):
        t['attention_mask'][i][:lens[i]] = 0
    return t

# Токенизация данных
dataset = dataset.map(
    format_and_tokenize,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'input_ids': tensor([[21106,   318,   281,  ..., 50256, 50256, 50256],
        [21106,   318,   281,  ..., 50256, 50256, 50256],
        [21106,   318,   281,  ..., 50256, 50256, 50256],
        ...,
        [21106,   318,   281,  ..., 50256, 50256, 50256],
        [21106,   318,   281,  ..., 50256, 50256, 50256],
        [21106,   318,   281,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [6]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling, default_data_collator

# Data collator to handle padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="./results",
    # do_train=False,
    # do_eval=True,
    per_device_eval_batch_size=1,
    per_device_train_batch_size=1,       
    gradient_accumulation_steps=1, 
    fp16=True,
    report_to="none",
    eval_accumulation_steps=10
)


# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    eval_dataset=dataset,  # Используем подмножество для демонстрации
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_208926/1012142508.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [7]:
# trainer.train()

In [8]:
eval_results = trainer.evaluate()
print(eval_results)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


(10, 768) (10, 768)
{'accuracy': 0.10408921933085502}
{'accuracy': 0.125}
{'accuracy': 0.13602941176470587}
{'accuracy': 0.043859649122807015}
{'accuracy': 0.10869565217391304}
{'accuracy': 0.09969788519637462}
{'accuracy': 0.13636363636363635}
{'accuracy': 0.10285714285714286}
{'accuracy': 0.08812260536398467}
{'accuracy': 0.04838709677419355}
{'eval_loss': 6.796446800231934, 'eval_model_preparation_time': 0.0054, 'eval_accuracy': 0.0993102298947613, 'eval_runtime': 2.4109, 'eval_samples_per_second': 4.148, 'eval_steps_per_second': 4.148}


In [9]:
# from transformers import pipeline

# # pl = pipeline(
# #     "text-generation",
# #     model=model,
# #     tokenizer=tokenizer,
# #     torch_dtype=torch.float16
# # )

# # with torch.inference_mode(), torch.cuda.amp.autocast():
# #     print(pl('Hello ', max_new_tokens=16,))

# pl = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.float16
# )

# with torch.inference_mode(), torch.cuda.amp.autocast():
#     print(pl('Hello ', max_new_tokens=16, do_sample=False, temperature=None, top_p=None,))

In [10]:
# import torch
# import gc
# torch.cuda.empty_cache()
# gc.collect()
# from numba import cuda
 
# cuda.select_device(1) # choosing second GPU 
# cuda.close()

In [11]:
for e in trainer.train_dataset:
    print(len(e['input_ids']))

TypeError: 'NoneType' object is not iterable